In [2]:
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import ARRAY, TEXT
from sqlalchemy import create_engine, text
import pandas as pd
import ast


In [ ]:
USER = "postgres"
PASSWORD = "#####"
HOST = "127.0.0.1"
PORT = "5432"
DB_NAME = "Trabalho-DW"
engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")


Converte Preco para float

In [12]:
df = pd.read_csv("../quintoAndarTratado.csv")

# Passo 1: Remover os pontos que separam os milhares
df['Preco'] = df['Preco'].str.replace('.', '', regex=False)

# Passo 2: Substituir a vírgula por ponto para o separador decimal
df['Preco'] = df['Preco'].str.replace(',', '.', regex=False)

# Passo 3: Converter para float
df['Preco'] = pd.to_numeric(df['Preco'], errors='coerce')

#### Criando a tabela de dados gerais e inserindo os dados

In [9]:
# Comando SQL para criar a tabela
create_table_query = """
CREATE TABLE IF NOT EXISTS dados_gerais (
    id SERIAL PRIMARY KEY,
    preco FLOAT,
    num_quartos INT,
    num_banheiros INT,
    num_vagas INT,
    tamanho FLOAT,
    tipo TEXT,
    cidade TEXT,
    regiao TEXT,
    itens_disponiveis TEXT[],
    itens_indisponiveis TEXT[],
    data_completa TEXT,
    dia INT,
    mes INT,
    ano INT,
    dia_da_semana INT,
    dia_da_semana_extenso TEXT,
    semana_do_ano INT,
    quarter TEXT,
    estado TEXT
);
"""

# Executa o comando SQL para criar a tabela
with engine.connect() as connection:
    connection.execute(text(create_table_query))

print("Tabela 'dados_gerais' criada com sucesso!")

Tabela 'dados_gerais' criada com sucesso!


In [14]:
# Função para adaptar os itens para o postgres suportar
def convert_string_to_list(x):
    if isinstance(x, str):
        x = x.strip()
        if not x.startswith('['):
            x = '[' + x + ']'
        try:
            lista = ast.literal_eval(x)
            return [str(item).strip() for item in lista]
        except Exception:
            items = x.strip('[]').split(',')
            return [item.strip(" '\"") for item in items if item.strip(" '\"")]
    return []


# Renomeia as colunas para igualar aos da tabela
df.rename(columns={
    "Preco": "preco",
    "Num Quartos": "num_quartos",
    "Num Banheiros": "num_banheiros",
    "Num Vagas": "num_vagas",
    "Tamanho": "tamanho",
    "Tipo": "tipo",
    "Cidade": "cidade",
    "Regiao": "regiao",
    "Data Completa": "data_completa",
    "Dia": "dia",
    "Mes": "mes",
    "Ano": "ano",
    "Dia da Semana": "dia_da_semana",
    "Dia da Semana Extenso": "dia_da_semana_extenso",
    "Semana do Ano": "semana_do_ano",
    "Quarter": "quarter",
    "Estado": "estado",
    "Itens Disponivels": "itens_disponiveis", 
    "Itens Indisponíveis": "itens_indisponiveis"
}, inplace=True)

# Aplica a conversão nas colunas de itens
df["itens_disponiveis"] = df["itens_disponiveis"].apply(convert_string_to_list)
df["itens_indisponiveis"] = df["itens_indisponiveis"].apply(convert_string_to_list)

# Mapeamento de tipos para as colunas que são arrays
dtype_mapping = {
    "itens_disponiveis": ARRAY(TEXT),
    "itens_indisponiveis": ARRAY(TEXT)
}

# Insere os dados na tabela
df.to_sql("dados_gerais", engine, if_exists="append", index=False, dtype=dtype_mapping)



720

#### Agora com os dados inseridos na tabela do postgres, o resto do projeto será feito diretamente no Query tool do postgres, sendo o notebook usado apenas para a documentação e apresentação

# Esquema do DW

![Esquema do Data Warehouse](./Fotos/esquemaDW.png)

## Dimensão Tempo

#### Criação da dimensão tempo

In [ ]:
CREATE TABLE dimensao_tempo (
    id_tempo SERIAL PRIMARY KEY,
    dia INT,
    mes INT,
    Ano INT,
    data_completa TEXT,
    dia_da_semana INT,
    dia_da_semana_extenso TEXT,
    semana_do_ano INT,
    quarter TEXT
);

#### Povoando a dimensao tempo

In [ ]:
INSERT INTO dimensao_tempo (
    dia, 
    mes, 
    ano, 
    data_completa, 
    dia_da_semana, 
    dia_da_semana_extenso, 
    semana_do_ano, 
    quarter
)
SELECT 
    dia, 
    mes, 
    ano, 
    data_completa, 
    dia_da_semana, 
    dia_da_semana_extenso, 
    semana_do_ano, 
    quarter
FROM dados_gerais
GROUP BY 
    dia, mes, ano, data_completa, 
    dia_da_semana, dia_da_semana_extenso, 
    semana_do_ano, quarter;


## Dimensão Região

#### Criando a dimensão Região

In [ ]:
CREATE TABLE dimensao_regiao (
	id_regiao SERIAL PRIMARY KEY,
	estado TEXT,
	cidade TEXT,
	regiao TEXT
)

### Povoando a dimensão região

In [ ]:
INSERT INTO dimensao_regiao (estado, cidade, regiao)
SELECT estado, cidade, regiao 
FROM dados_gerais
GROUP BY estado, cidade, regiao;

## Dimensao Item

### Criando a dimensão

In [ ]:
CREATE TABLE dimensao_item (
	id_item SERIAL PRIMARY KEY,
	item TEXT
)

### Povoando a dimensão item

In [ ]:
INSERT INTO dimensao_item (item)
SELECT DISTINCT item
FROM (
    SELECT unnest(itens_disponiveis) AS item
    FROM dados_gerais
    UNION
    SELECT unnest(itens_indisponiveis) AS item
    FROM dados_gerais
) AS todos_itens;


## Dimensão Imóvel

### Criando a dimensão

In [ ]:
CREATE TABLE dimensao_imovel (
	id_imovel SERIAL PRIMARY KEY,
	tipo TEXT,
	itens_disponiveis INT[] NOT NULL,
	itens_indisponiveis INT[] NOT NULL
)

### Populando a dimensão

In [ ]:
INSERT INTO dimensao_imovel (tipo, itens_disponiveis, itens_indisponiveis)
SELECT DISTINCT
    dg.tipo,
    ARRAY(
        SELECT id_item 
        FROM dimensao_item 
        WHERE item = ANY(dg.itens_disponiveis)
    ),
    ARRAY(
        SELECT id_item 
        FROM dimensao_item 
        WHERE id_item NOT IN (
            SELECT id_item 
            FROM dimensao_item 
            WHERE item = ANY(dg.itens_disponiveis)
        )
    )
FROM dados_gerais dg;

## Dimensão característica
#### Criada pois havia muitas amostras que compartilhavam todas as caracterśticas, com exceção do preço

### Criando a dimensao

In [ ]:
CREATE TABLE dimensao_caracteristicas (
	id_caracteristicas SERIAL PRIMARY KEY,
	tamanho float,
	num_quartos int,
	num_banheiros int, 
	num_vagas int
)

### Populando a dimensao

In [ ]:
INSERT INTO dimensao_caracteristicas (tamanho, num_quartos, num_banheiros, num_vagas)
SELECT DISTINCT
    dg.tamanho::FLOAT,
    dg.num_quartos,
    dg.num_banheiros,
    dg.num_vagas
FROM dados_gerais dg;


## Tabela Fato

### Criando a tabela

In [ ]:
CREATE TABLE fato_vendas (
    id_regiao INT NOT NULL,
    id_imovel INT NOT NULL,
    id_tempo INT NOT NULL,
    id_caracteristicas INT NOT NULL,
    preco NUMERIC(10,2),  -- Usando um tipo adequado para valores monetários

    -- Definição de chaves estrangeiras
    FOREIGN KEY (id_regiao) REFERENCES dimensao_regiao(id_regiao),
    FOREIGN KEY (id_tempo) REFERENCES dimensao_tempo(id_tempo),
    FOREIGN KEY (id_imovel) REFERENCES dimensao_imovel(id_imovel),
    FOREIGN KEY (id_caracteristicas) REFERENCES dimensao_caracteristicas(id_caracteristicas),

    -- Chave primária composta para garantir unicidade
    PRIMARY KEY (id_regiao, id_imovel, id_tempo, id_caracteristicas)
);


### Populando a tabela fato

In [ ]:
INSERT INTO fato_vendas (
    id_regiao,
    id_imovel,
    id_tempo,
    id_caracteristicas,
    preco
)
SELECT 
    dr.id_regiao,
    di.id_imovel,
    dt.id_tempo,
    dc.id_caracteristicas,
    dg.preco::NUMERIC(10,2)
FROM 
    dados_gerais dg
INNER JOIN dimensao_regiao dr 
    ON dg.estado = dr.estado 
    AND dg.cidade = dr.cidade 
    AND dg.regiao = dr.regiao
INNER JOIN dimensao_tempo dt 
    ON dg.data_completa = dt.data_completa
INNER JOIN dimensao_imovel di 
    ON dg.tipo = di.tipo
    AND (
         SELECT ARRAY(
             SELECT id_item 
             FROM dimensao_item 
             WHERE item = ANY(dg.itens_disponiveis)
             ORDER BY id_item
         )
    ) = (
         SELECT ARRAY(
             SELECT unnest(di.itens_disponiveis)
             ORDER BY 1
         )
    )
INNER JOIN dimensao_caracteristicas dc
    ON dg.tamanho::FLOAT = dc.tamanho
    AND dg.num_quartos = dc.num_quartos
    AND dg.num_banheiros = dc.num_banheiros
    AND dg.num_vagas = dc.num_vagas;
